In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import re
import ntpath
import pandas as pd
pd.options.display.max_rows = 150

In [ ]:
def save_fig(filename, folder, trans=True, dpi=800, bbox_inches='tight', pad=0.1):
    if not os.path.exists(folder):
        os.makedirs(folder)
    plt.savefig('{}/{}.png'.format(folder, filename), transparent=trans, dpi=dpi, bbox_inches=bbox_inches, pad_inches=pad) 
    return 

In [ ]:
def rebin(data, rebin_ratio, do_average=False):
    """ Rebin a 1D array the good old way.
    @param 1d numpy array data : The data to rebin
    @param int rebin_ratio: The number of old bin per new bin
    @return 1d numpy array : The array rebinned
    The last values may be dropped if the sizes do not match. """
    
    rebin_ratio = int(rebin_ratio)
    length = (len(data) // rebin_ratio) * rebin_ratio
    data = data[0:length]
    data = data.reshape(length//rebin_ratio, rebin_ratio)
    if do_average :
        data_rebinned = data.mean(axis=1)
    else :
        data_rebinned = data.sum(axis=1)
    return data_rebinned

def decimate(data, decimation_ratio):
    """ Decimate a 1D array . This means some value are dropped, not averaged
    @param 1d numpy array data : The data to decimated
    @param int decimation_ratio: The number of old value per new value
    @return 1d numpy array : The array decimated. """
    
    decimation_ratio = int(decimation_ratio)
    length = (len(data) // decimation_ratio) * decimation_ratio
    data_decimated = data[:length:decimation_ratio]
    return data_decimated

def rebin_xy(x, y, ratio=1, do_average=True):
    """ Helper method to decimate x and rebin y, with do_average True as default. """
    return decimate(x, ratio), rebin(y, ratio, do_average)

In [ ]:
# Spectrometer offset on the grating at 75 lines
offset_spectro = 0.018

In [ ]:
def get_all_data_folders(search_str='', folder=None, file_format='', print_info=False):
    """Search in a folder and its subfolders all the files containing a given string in their name or filepath.

    @param string search_str (optional): the string to search in the file name and filepath
    @param string folder (optional): the folder to search into
    @param string file_format (optional): by default return all the file format, or else specify the format like 'dat' or '.dat'
    @param string print_info (optional): print the number of found files if true

    @return list : the list of all folders in which at least a data file has been found.

    """

    if folder == None:
        search_dir = os.getcwd()
    else:
        search_dir = folder

    valid_folders = []

    if len(file_format) == 0:
        for (dirpath, dirnames, files) in os.walk(search_dir):
            for name in files:
                if (search_str in name) & (dirpath not in valid_folders):
                    valid_folders.append(dirpath)
            for dirname in dirnames:
                if search_str in dirname:
                    valid_folders.append(os.path.join(dirpath, dirname))
    else:
        m = -len(file_format)
        for (dirpath, dirnames, files) in os.walk(search_dir):
            for name in files:
                if (search_str in name) & (name[m:] == file_format) & (dirpath not in valid_folders):
                    valid_folders.append(dirpath)
            for dirname in dirnames:
                if (search_str in dirname):
                    for name in os.listdir(os.path.join(dirpath, dirname)):
                        if (name[m:] == file_format) & (os.path.join(dirpath, dirname) not in valid_folders):
                            valid_folders.append(os.path.join(dirpath, dirname))

    if print_info:
        print(len(valid_folders), 'folder(s) found in ', search_dir)

    return valid_folders

In [ ]:
path_all = get_all_data_folders(search_str='202308', folder=r'C:\Users\YB274940\Desktop\Data_run14', file_format='.txt', print_info=False)
path_all

In [ ]:
path = r'C:\Users\YB274940\Desktop\Data_run14\20230829'
All_files = os.listdir(path)

dictionary = {}
save_name_list = []
frames = []

for file in All_files:
    if file.endswith('.txt'):  # Check if the file has a .txt extension
        file_path = os.path.join(path, file)  # Create the full file path
        pattern = "(.*?).txt"
        save_name = re.search(pattern, file).group(1)
        save_name_list.append(save_name)
        df = pd.read_csv(path + '\\' + file, sep='	', skiprows=0)
        
        frames.append(pd.Series({'wavelength': df.iloc[:,0], 'pl_intensity': df.iloc[:,1], 'filename': ntpath.basename(file)}).to_frame().transpose())
    data = pd.concat(frames, sort=False).reset_index(drop=True)

In [ ]:
data

# Simple spectrum

In [ ]:
# filename = r"C:\Users\YB274940\Desktop\Data_run14\20230828\20230828_Q4E_0p8mW.txt" # Q4E
filename = r"C:\Users\YB274940\Desktop\Data_run14\20230828\20230828_S1A_0p8mW.txt" # S1A
# filename = r"S:\120-LETI\120.21-DPFT\120.21.14-SSURF\120.21.14.2-Info_communes\WTeq\Data_run14\20230829\20230829_T2A_Punknown.txt"

# filename = r"S:\120-LETI\120.21-DPFT\120.21.14-SSURF\120.21.14.2-Info_communes\WTeq\Data_run14\20230830\20230830_T2B_f50um.txt"
# filename = r"S:\120-LETI\120.21-DPFT\120.21.14-SSURF\120.21.14.2-Info_communes\WTeq\Data_run14\20230830\20230830_T2C_f50um.txt"

# filename = r"S:\120-LETI\120.21-DPFT\120.21.14-SSURF\120.21.14.2-Info_communes\WTeq\Data_run14\20230831\20230831_T3D.txt"

In [ ]:
# IMPORT DATA
data = pd.read_csv(filename, sep='	', skiprows=0)
data['Wavelength_u'] = data['Wavelength']/1e3
Wavelength = np.array(data["Wavelength"], dtype=float)
S1 = np.array(data["S1"], dtype=float)

x_min, x_max = data["Wavelength"].min()/1e3-offset_spectro, data["Wavelength"].max()/1e3-offset_spectro
y_min, y_max = 0,(data["S1"].max() + data["S1"].max()*20/100)/1e3

# PLOT
fig, ax = plt.subplots(figsize=(8, 4))
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xlabel("Wavelength [$\mu m$]")
ax.set_ylabel("PL [kc/s]")
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.plot(data["Wavelength_u"]-offset_spectro, data["S1"]/1e3, marker='None', c="dodgerblue")
ax.fill_between(Wavelength/1e3-offset_spectro, S1/1e3, color='dodgerblue', alpha=0.2)
ax.axvline(x=1.218, color='black', ls='--', lw=0.75, alpha=0.4)

# ax.fill_between([0.94,1.2], 2, color='crimson', alpha=0.05)
# ax.text(0.98, y_max-y_max*0.1, 'Filter', color='darkred', fontweight='bold', alpha=0.7)

# ax.text(0.98, y_max-y_max*0.1, 'Q4E sample', color='forestgreen', fontweight='bold')
# ax.text(0.98, y_max-y_max*0.1, 'S1A sample', color='forestgreen', fontweight='bold')
# ax.text(0.98, y_max-y_max*0.1, 'T2B sample', color='forestgreen', fontweight='bold')
# ax.text(0.98, y_max-y_max*0.1, 'T2C sample', color='forestgreen', fontweight='bold')
# ax.text(0.98, y_max-y_max*0.1, 'T2A sample', color='forestgreen', fontweight='bold')
# ax.text(1.24, y_max-y_max*0.1, 'T3D sample', color='forestgreen', fontweight='bold')
pass

# Double spectra

In [ ]:
# filename = r"C:\Users\YB274940\Desktop\Data_run14\20230828\20230828_T1A_0p8mW.txt" # T1A (28/08/2023)
# filename_zoom = r"C:\Users\YB274940\Desktop\Data_run14\20230828\20230828_T1A_0p8mW_900tr.txt" # T1A (28/08/2023)

filename = r"S:\120-LETI\120.21-DPFT\120.21.14-SSURF\120.21.14.2-Info_communes\WTeq\Data_run14\20230830\20230830_T2D_f50um.txt"
filename_zoom = r"S:\120-LETI\120.21-DPFT\120.21.14-SSURF\120.21.14.2-Info_communes\WTeq\Data_run14\20230830\20230830_T2D_f50um_900tr.txt"

In [ ]:
data = pd.read_csv(filename, sep='	', skiprows=0)
data_zoom = pd.read_csv(filename_zoom, sep='	', skiprows=0)

data['Wavelength_u'] = data['Wavelength']/1e3
Wavelength = np.array(data["Wavelength"], dtype=float)
S1 = np.array(data["S1"], dtype=float)

data_zoom['Wavelength_u'] = data_zoom['Wavelength']/1e3
Wavelength_zoom = np.array(data_zoom["Wavelength"], dtype=float)
S1_zoom = np.array(data_zoom["S1"], dtype=float)

# PLOT (75 lines)
fig, ax = plt.subplots(figsize=(8, 4))
ax.set_xlim(data["Wavelength"].min()-offset_spectro*1e3+120
            , data["Wavelength"].max()-offset_spectro*1e3)
ax.set_ylim(0,(data["S1"].max() + data["S1"].max()*2/100)/1e3)
ax.set_xlabel("Wavelength [$nm$]")
ax.set_ylabel("PL [kc/s]")
ax.xaxis.set_major_locator(plt.MaxNLocator(9))

ax.plot(data["Wavelength"]-offset_spectro*1e3, data["S1"]/1e3, marker='None', c="dodgerblue", label='T2D sample')
ax.fill_between(Wavelength-offset_spectro*1e3, S1/1e3, color='dodgerblue', alpha=0.2)
ax.fill_between([data_zoom["Wavelength"].min(), data_zoom["Wavelength"].max()], 10, color='crimson', alpha=0.05)
ax.axvline(x=1218, color='gray', ls='--', lw=0.75, label='W-center ZPL')

ax.text(1225, 2.5, 'W-center ZPL', color='gray', fontweight='bold')
ax.text(1140, 8.5, 'T2D sample', color='dodgerblue', fontweight='bold')

# Inset (900 lines) #

ax2 = plt.axes([0.56, 0.5, 0.32, 0.35])

ax2.set_xlim(data_zoom["Wavelength"].min(), data_zoom["Wavelength"].max())
ax2.set_ylim(0,(data_zoom["S1"].max() + data_zoom["S1"].max()*2/100))
ax2.set_xlabel("Wavelength [$nm$]")
ax2.set_ylabel("PL [c/s]")
ax2.xaxis.set_major_locator(ticker.FixedLocator([1210, 1220, 1230]))

ax2.plot(data_zoom["Wavelength"], data_zoom["S1"], marker='None', c="crimson", label='T2D sample')
ax2.fill_between(Wavelength_zoom, S1_zoom, color='crimson', alpha=0.2)
ax2.axvline(x=1218, color='gray', ls='--', lw=0.75, label='W-center ZPL')
ax2.text(1203, 300, 'Zoom on ZPL', color='crimson', fontweight='bold')
ax2.text(1202.5, 270, '(using high-res grating)', color='crimson', fontsize=7)
# ax.legend(loc=2, fontsize=10)
# ax2.legend(loc=2, fontsize=8)

folder = r'C:\Users\YB274940\Desktop'
save_fig('T2D', folder, dpi=800, bbox_inches='tight', pad=0.1)
pass

# Broad Spectrum

In [ ]:
filename = r'C:\Users\YB274940\Desktop\Data_run14\20230828\20230828_T1A_0p8mW_900tr_broad.txt' #T1A HD (28/08/2023)

In [ ]:
data = pd.read_csv(filename, sep='	', skiprows=0)
data['Wavelength_u'] = data['Wavelength']/1e3
Wavelength_u = np.array(data["Wavelength_u"], dtype=float)
S1 = np.array(data["S1"], dtype=float)
x_to_plot, y_to_plot = rebin_xy(Wavelength_u, S1, ratio=2)
x_min, x_max = data["Wavelength"].min()/1e3, data["Wavelength"].max()/1e3
# y_min, y_max = 0,(data["S1"].max() + data["S1"].max()*2/100)/1e3
y_min, y_max = 0.01,(data["S1"].max() + data["S1"].max()*20/100)/1e3

# PLOT
fig, ax = plt.subplots(figsize=(8, 4))
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xlabel("Wavelength [$\mu m$]")
ax.set_ylabel("PL [kc/s]")
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.plot(x_to_plot, y_to_plot/1e3, marker='None', c="dodgerblue")
ax.fill_between(x_to_plot, y_to_plot/1e3, color='dodgerblue', alpha=0.2)

ax.axvline(x=1.218, color='black', ls='--', lw=0.75, alpha=0.4)
# ax.text(1.0, y_max-y_max*0.4, 'T1A sample', color='forestgreen', fontweight='bold')
ax.set_yscale('log')
pass

In [ ]:
import os
import pandas as pd
import ntpath
import numpy as np

# Specify the main directory where your data folders are located
main_directory = r'C:\Users\YB274940\Desktop\Data_run14'

# Get a list of all subdirectories in the main directory
subdirectories = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

# Create an empty list to store DataFrames
dfs = []

# Loop through each subdirectory and read the CSV files into DataFrames
for subdirectory in subdirectories:
    folder_path = os.path.join(main_directory, subdirectory)
    
    # Assuming all CSV files have the same structure, adjust as needed
    for file in os.listdir(folder_path):
        if file.endswith('.txt'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path, sep='	', skiprows=0)
            dfs.append(pd.Series({'wavelength': np.array(df.iloc[:,0]), 'pl_intensity': np.array(df.iloc[:,1]), 'filename': ntpath.basename(file)}).to_frame().transpose())

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

In [ ]:
# final_df

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))

j=0
for i, row in final_df.iterrows():
    if len(row.wavelength) == 256:
        if '900tr' not in row.filename:
            if '_S' in row.filename:
                if '1200' in row.filename:
                    ax.plot(row.wavelength, row.pl_intensity+j*5e3, marker='None', label=row.filename[9:-22])
                    j += 1
ax.set_ylim(0, 50e3)
ax.legend(ncol=3)
pass

In [ ]:
# Assuming you've already filtered the DataFrame as shown in the previous example
given_string = '_T'
filtered_df = final_df[final_df['filename'].str.contains(given_string)]

# Iterate through the rows and print information
for index, row in filtered_df.iterrows():
    if '900tr' not in row.filename:
        filename = row['filename']
        wavelength_length = len(row['wavelength'])

        print(f"Filename: {filename}, Wavelength Length: {wavelength_length}")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))

j=0
for i, row in final_df.iterrows():
    if len(row.wavelength) == 256:
        if '900tr' not in row.filename:
            if '_T' in row.filename:
                # if '1200' in row.filename:
                ax.plot(row.wavelength, row.pl_intensity+j*5e3, marker='None', label=row.filename[9:-4])
                j += 1
# ax.set_ylim(0, 50e3)
ax.legend(ncol=3)
pass